In [1]:
import numpy as np
from tensorflow.keras import optimizers,Sequential, layers, Model
import matplotlib.pyplot as plt
import tensorflow as tf

In [2]:
# 一维卷积后，再全连接获得三层预测
class FcCvModelReFCSpeed(Model):
    def __init__(self):
        super(FcCvModelReFCSpeed, self).__init__()
        # 多层全链接的第一层，每一道数据的提取
        # 实际上是在每道上的滑动全连接
        self.fcOne = Sequential([
            layers.Dense(64),
            layers.BatchNormalization(),
            layers.LeakyReLU(0.2),
            layers.Dense(8),
            layers.BatchNormalization(),
            layers.LeakyReLU(0.2)
        ])
        # 1.减少过拟合的情况
        # 2.模拟道数据的丢失
        self.dropout_fc = layers.Dropout(0.3)

        # 1维卷积综合提取特征
        self.conv1 = Sequential([
            layers.Conv1D(filters=64, kernel_size=9, strides=1, padding='same'),
            layers.BatchNormalization(),
            layers.LeakyReLU(0.2)
        ])
        self.dropout_c1 = layers.Dropout(0.3)
        self.conv2 = Sequential([
            layers.Conv1D(filters=64, kernel_size=9, strides=1, padding='same'),
            layers.BatchNormalization(),
            layers.LeakyReLU(0.2)
        ])
        self.dropout_c2 = layers.Dropout(0.3)
        # 1维卷积综合提取特征+降维
        self.conv3 = Sequential([
            layers.Conv1D(filters=128, kernel_size=4, strides=4, padding='valid'),
            layers.BatchNormalization(),
            layers.LeakyReLU(0.2)
        ])
        self.dropout_c3 = layers.Dropout(0.3)
        # 1维卷积综合提取特征
        self.conv4 = Sequential([
            layers.Conv1D(filters=128, kernel_size=7, strides=1, padding='same'),
            layers.BatchNormalization(),
            layers.LeakyReLU(0.2)
        ])
        self.dropout_c4 = layers.Dropout(0.3)
        self.conv5 = Sequential([
            layers.Conv1D(filters=128, kernel_size=7, strides=1, padding='same'),
            layers.BatchNormalization(),
            layers.LeakyReLU(0.2)
        ])
        self.dropout_c5 = layers.Dropout(0.3)
        # 1维卷积综合提取特征+降维
        self.conv6 = Sequential([
            layers.Conv1D(filters=256, kernel_size=4, strides=4, padding='valid'),
            layers.BatchNormalization(),
            layers.LeakyReLU(0.2)
        ])
        
        # 最终作出的预测
        # 速度预测
        self.fcThree_s1 = Sequential([
            layers.Dense(8),
            layers.BatchNormalization(),
            layers.Activation('softmax')
        ])
        self.fcThree_s2 = Sequential([
            layers.Dense(8),
            layers.BatchNormalization(),
            layers.Activation('softmax')
        ])
        self.fcThree_s3 = Sequential([
            layers.Dense(8),
            layers.BatchNormalization(),
            layers.Activation('softmax')
        ])
        self.fcThree_s4 = Sequential([
            layers.Dense(8),
            layers.BatchNormalization(),
            layers.Activation('softmax')
        ])

    def call(self, inputs, training=None, mask=None):
        batchSize, dao_num, data_num = inputs.shape
        x = tf.reshape(inputs, (-1, 1024))
        x = self.fcOne(x)
        x = tf.reshape(x, (batchSize, dao_num, -1))
        x = tf.transpose(x, perm=[0, 2, 1])
        x = self.dropout_fc(x, training=training)
        x = self.conv1(x)
        x = self.dropout_c1(x, training=training)
        x = self.conv2(x)
        x = self.dropout_c2(x, training=training)
        x = self.conv3(x)
        x = self.dropout_c3(x, training=training)
        x = self.conv4(x)
        x = self.dropout_c4(x, training=training)
        x = self.conv5(x)
        x = self.dropout_c5(x, training=training)
        x = self.conv6(x)
        x = tf.reshape(x, (batchSize, -1))
        x_s1 = self.fcThree_s1(x)
        x_s2 = self.fcThree_s2(x)
        x_s3 = self.fcThree_s3(x)
        x_s4 = self.fcThree_s4(x)
        x_s = tf.concat([tf.expand_dims(x_s1,axis=1),tf.expand_dims(x_s2,axis=1),tf.expand_dims(x_s3,axis=1),tf.expand_dims(x_s4,axis=1)],axis=1)
        return x_s

In [3]:
loss_log_path = './tf_dir/loss_all_layer3_fc_speed_2'
loss_summary_writer = tf.summary.create_file_writer(loss_log_path)

In [4]:
train_loss_speed_m = tf.keras.metrics.Mean('train_loss_speed', dtype=tf.float32)
train_acc_m = tf.keras.metrics.Mean('train_acc', dtype=tf.float32)
dev_loss_speed_m = tf.keras.metrics.Mean('dev_loss_speed', dtype=tf.float32)
dev_acc_m = tf.keras.metrics.Mean('dev_acc', dtype=tf.float32)

In [5]:
x = np.load('/openbayes/input/input0/data_all.npy')
# y = np.load('/openbayes/input/input1/label_time.npy')
y_class = np.load('/openbayes/input/input2/label_class.npy')
y_class_oh = tf.one_hot(y_class.astype(np.int32),depth=8,axis=-1)

In [6]:
optimizer = optimizers.Adam(lr=4e-5)
epochs = 10000
loss_speed_p = 1000
model = FcCvModelReFCSpeed()

In [7]:
model.load_weights('weights_fc_speed/weights_'+str(5801))

In [8]:
# y = y * 1000

In [ ]:
for e in range(epochs,epochs+10000):
    for i in range(12):
#         for step in range(2):[step*256:(step+1)*256]
        with tf.GradientTape() as tape:
            y_speed_pred = model(x[i*512:(i+1)*512],training=True)
            train_loss_speed = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_speed_pred, y_class_oh[i*512:(i+1)*512]))
        grads = tape.gradient(train_loss_speed, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))
        train_loss_speed_m(train_loss_speed)
        y_speed_pred_ = tf.argmax(y_speed_pred,axis=-1)
        train_acc = tf.reduce_mean(tf.cast(y_speed_pred_ == tf.constant(y_class[i*512:(i+1)*512],dtype=tf.int64),dtype=tf.float32))
        train_acc_m(train_acc)
    with loss_summary_writer.as_default():
        tf.summary.scalar('train_loss_speed', train_loss_speed_m.result(), step=e)
        tf.summary.scalar('train_acc', train_acc_m.result(), step=e)
    train_loss_speed_m.reset_states()
    train_acc_m.reset_states()
    if e%100 == 0:
        for i in range(4):
            y_dev_speed_pred = model(x[(i+12)*512:(i+16)*512],training=False)
            dev_loss_speed = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_dev_speed_pred, y_class_oh[(i+12)*512:(i+16)*512]))
            dev_loss_speed_m(dev_loss_speed)
            y_dev_speed_pred_ = tf.argmax(y_dev_speed_pred,axis=-1)
            dev_acc = tf.reduce_mean(tf.cast(y_dev_speed_pred_ == tf.constant(y_class[(i+12)*512:(i+16)*512],dtype=tf.int64),dtype=tf.float32))
            dev_acc_m(dev_acc)
        with loss_summary_writer.as_default():
            tf.summary.scalar('dev_loss_speed', dev_loss_speed_m.result(), step=e)
            tf.summary.scalar('dev_acc', dev_acc_m.result(), step=e)
        dev_loss_speed_m.reset_states()
        dev_acc_m.reset_states()
        model.save_weights('weights_fc_speed_2/weights_'+str(e+1))